# Classes

For more information on the magic methods of pytho classes, consult the docs: https://docs.python.org/3/reference/datamodel.html


In [1]:
class DumbClass:
    """ This class is just meant to demonstrate the magic __repr__ method
    """
    
    def __repr__(self):
        """ I'm giving this method a docstring
        """
        return("I'm representing an instance of my dumbclass")

dc = DumbClass()
print(dc)
dc

I'm representing an instance of my dumbclass


I'm representing an instance of my dumbclass

In [2]:
help(DumbClass)

Help on class DumbClass in module __main__:

class DumbClass(builtins.object)
 |  This class is just meant to demonstrate the magic __repr__ method
 |  
 |  Methods defined here:
 |  
 |  __repr__(self)
 |      I'm giving this method a docstring
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



In [3]:
class Stack:
    """ A simple class implimenting some common features of Stack
        objects
    """
    
    def __init__(self, iterable=None):
        """ Initializes Stack objects. If an iterable is provided,
            add elements from the iterable to this Stack until the
            iterable is exhausted
        """
        self.head = None
        self.size = 0
        if(iterable is not None):
            for item in iterable:
                self.add(item)
    
    def add(self, item):
        """ Add an element to the top of the stack. This method will
            modify self and return self.
        """
        self.head = (item, self.head)
        self.size += 1
        return self
    
    def pop(self):
        """ remove the top item from the stack and return it
        """
        if(len(self) > 0):
            ret = self.head[0]
            self.head = self.head[1]
            self.size -= 1
            return ret
        return None
    
    def __contains__(self, item):
        """ Returns True if item is in self
        """
        for i in self:
            if(i == item):
                return True
        return False
    
    def __len__(self):
        """ Returns the number of items in self
        """
        return self.size
    
    def __iter__(self):
        """ prepares this stack for iteration and returns self
        """
        self.curr = self.head
        return self
    
    def __next__(self):
        """ Returns items from the stack from top to bottom
        """
        if(not hasattr(self, 'curr')):
            iter(self)
        if(self.curr is None):
            raise StopIteration
        else:
            ret = self.curr[0]
            self.curr = self.curr[1]
            return ret
    
    def __reversed__(self):
        """ returns a copy of self with the stack turned upside
            down
        """
        return Stack(self)
    
    
    def __add__(self, other):
        """ Put self on top of other
        """
        ret = Stack(reversed(other))
        for item in reversed(self):
            ret.add(item)
        return ret
    
    def __repr__(self):
        """ Represent self as a string
        """
        return f'Stack({str(list(self))})'

In [4]:
# Create a stack object and test some methods
x = Stack([3, 2])
print(x)

# adds an element to the top of the stack
print('\nLets add 1 to the stack')
x.add(1)
print(x)

# Removes the top most element
print('\nLets remove an item from the top of the stack')
item = x.pop()
print(item)
print(x)

# Removes the top most element
print('\nlets remove another item')
item = x.pop()
print(item)
print(x)

Stack([2, 3])

Lets add 1 to the stack
Stack([1, 2, 3])

Lets remove an item from the top of the stack
1
Stack([2, 3])

lets remove another item
2
Stack([3])


In [5]:
x = Stack([4,5,6])
# Because I implimented the __contains__ method,
# I can check if items are in stack objects
print(f'Does my stack contain 2? {2 in x}')
print(f'Does my stack contain 4? {4 in x}')
# Because I implimented the __len__ method,
# I can check how many items are in stack objects
print(f'How many elements are in my stack? {len(x)}')

Does my stack contain 2? False
Does my stack contain 4? True
How many elements are in my stack? 3


In [ ]:
# because my stack class has an __iter__ and __next__ methods
# I can iterate over stack objects
x = Stack([7,3,4])
print(f"Lets iterate over my stack : {x}")
for item in x:
    print(item)
# Because my stack class has a __reversed__ method,
# I can easily reverse a stack object
print(f'I am flipping my stack upside down : {reversed(x)}')

In [6]:
# Because I implimented the __add__ method,
# I can add stacks together
x = Stack([4,5,6])
y = Stack([1,2,3])
print("I have two stacks")
print(f'x : {x}')
print(f'y : {y}')
print("Let's add them together")
print(f'x + y = {x + y}')
for item in (x + y):
    print(item)

I have two stacks
x : Stack([6, 5, 4])
y : Stack([3, 2, 1])
Let's add them together
x + y = Stack([6, 5, 4, 3, 2, 1])
6
5
4
3
2
1


# Using the SqlAlchemy ORM
For more information, check out the documentation : https://docs.sqlalchemy.org/en/latest/orm/tutorial.html

In [9]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float, ForeignKey
from sqlalchemy.orm import Session, relationship
import pymysql
pymysql.install_as_MySQLdb()

In [10]:
# Sets an object to utilize the default declarative base in SQL Alchemy
Base = declarative_base()


# Lets define the owners table/class
class Owners(Base):
    __tablename__ = 'owners'
    id = Column(Integer, primary_key=True)
    name = Column(String(255))
    phone_number = Column(String(255))
    pets = relationship("Pets", back_populates="owner")

    def __repr__(self):
        return f"<Owners(id={self.id}, name='{self.name}', phone_number='{self.phone_number}')>"


# Lets define the pets table/class
class Pets(Base):
    __tablename__ = 'pets'
    id = Column(Integer, primary_key=True)
    name = Column(String(255))
    owner_id = Column(Integer, ForeignKey('owners.id'))
    owner = relationship("Owners", back_populates="pets")
    
    def __repr__(self):
        return f"<Pets(id={self.id}, name='{self.name}', owner_id={self.owner_id})>"

In [12]:
# Lets connect to my database
# engine = create_engine("sqlite:///pets.sqlite")
engine = create_engine("mysql://root@localhost/review_db")
# conn = engine.connect()
Base.metadata.create_all(engine)
session = Session(bind=engine)

In [13]:
# Lets create me
me = Owners(name='Kenton', phone_number='867-5309')
session.add(me)
session.commit()
# Now lets add my dog
my_dog = Pets(name='Saxon', owner_id=me.id)
session.add(my_dog)
session.commit()

In [14]:
# We can query the tables using the session object from earlier
#  Lets just get all the data
all_owners = list(session.query(Owners))
all_pets = list(session.query(Pets))
print(all_owners)
print(all_pets)

[<Owners(id=1, name='Kenton', phone_number='867-5309')>]
[<Pets(id=1, name='Saxon', owner_id=1)>]


In [15]:
me = all_owners[0]
rio = all_pets[0]
# Because we are using an ORM and have defined relations,
# we can easily and intuitively access related data
print(me.pets)
print(rio.owner)

[<Pets(id=1, name='Saxon', owner_id=1)>]
<Owners(id=1, name='Kenton', phone_number='867-5309')>
